In [1]:
raw_data_dir = "/home/peterr/macocu/Varieties/BCMS"
import os
import parse

all_files = os.listdir(raw_data_dir)
p = parse.compile("setimes.{lang1}-{lang2}.{lang}.txt")

all_relevant_files = sorted([i for i in all_files if p.parse(i) is not None])


hr = 'setimes.bs-hr.hr.txt'
sr = 'setimes.bs-sr.sr.txt'
bs = 'setimes.bs-sr.bs.txt'

for file in [hr, sr, bs]:
    fullpath = os.path.join(raw_data_dir, file)
    wc = int(os.popen(f"wc -l <{fullpath}").read())

    print(file, wc, f"\n\tsplit 1: {int(wc*0.8)}, split 2: {int(wc*0.9)}")

setimes.bs-hr.hr.txt 138402 
	split 1: 110721, split 2: 124561
setimes.bs-sr.sr.txt 135945 
	split 1: 108756, split 2: 122350
setimes.bs-sr.bs.txt 135945 
	split 1: 108756, split 2: 122350


In [13]:
%%bash
sed -n '110721p' < ../../Varieties/BCMS/setimes.bs-hr.hr.txt

Kada je list The Economist pozvao birače na potporu oporbenoj CHP na nedavno održanim općim izborima u Turskoj, premijer Recep Tayyip Erdogan naznačio je da je za to možda kriv Izrael.


In [16]:
%%bash
sed -n '124561p' < ../../Varieties/BCMS/setimes.bs-hr.hr.txt

Razgovori su bili usmjereni na energetsku suradnju i jačanje odnosa.


In [19]:
%%bash
HRFILE=/home/peterr/macocu/Varieties/BCMS/setimes.bs-hr.hr.txt
SRFILE=/home/peterr/macocu/Varieties/BCMS/setimes.bs-sr.sr.txt
BSFILE=/home/peterr/macocu/Varieties/BCMS/setimes.bs-hr.bs.txt

INTERIM=/home/peterr/macocu/taskB/data/interim

# Croatian: 
sed -n '1,110715p' $HRFILE > "$INTERIM/hr_train.txt"
sed -n '110716,124548p' $HRFILE > "$INTERIM/hr_dev.txt"
sed -n '124549,$p' $HRFILE > "$INTERIM/hr_test.txt"

# Bosnian: 
sed -n '1,110715p' $BSFILE > "$INTERIM/bs_train.txt"
sed -n '110716,124548p' $BSFILE > "$INTERIM/bs_dev.txt"
sed -n '124549,$p' $BSFILE > "$INTERIM/bs_test.txt"

# Serbian: 
sed -n '1,108658p' $SRFILE > "$INTERIM/sr_train.txt"
sed -n '108659,122301p' $SRFILE > "$INTERIM/sr_dev.txt"
sed -n '122302,$p' $SRFILE > "$INTERIM/sr_test.txt"

In [40]:
interim_path = "/home/peterr/macocu/taskB/data/interim"


def count_separators(path: str, separator: str) -> int:
    ss = parse.compile(separator)
    with open(path, "r") as f:
        counter = 0
        for line in f.readlines():
            if ss.parse(line) is not None:
                counter += 1
                #print(line)
    return counter

separator = "{beginning}Southeast European Times{middle}{dd:d}{sep1}{mm:d}{sep2}{yy:d}{end}"

print("Searching for: ", separator)
print("""|filename | count |
|---|---|""")
for section in "train,dev,test".split(","):
    for language in "hr,bs,sr".split(","):
        filename = f"{language}_{section}.txt"
        input_path = os.path.join(interim_path, filename)
        
        count = count_separators(input_path, separator)
        print(f"|{filename}| {count}|")

Searching for:  {beginning}Southeast European Times{middle}{dd:d}{sep1}{mm:d}{sep2}{yy:d}{end}
|filename | count |
|---|---|
|hr_train.txt| 2557|
|bs_train.txt| 2557|
|sr_train.txt| 2261|
|hr_dev.txt| 326|
|bs_dev.txt| 326|
|sr_dev.txt| 308|
|hr_test.txt| 310|
|bs_test.txt| 309|
|sr_test.txt| 295|


In [21]:

count_separators("/home/peterr/macocu/taskB/data/interim/sr_test.txt", )

295

In [33]:
pattern = "{beginning}Southeast European Times{middle}{:d}{:d}/{:d}{:d}/{:d}{:d}"
line = "Svetla Dimitrova za Southeast European Times -- 12/01/12"

parse.parse(pattern, line)

<Result (1, 2, 0, 1, 1, 2) {'beginning': 'Svetla Dimitrova za ', 'middle': ' --'}>